In [ ]:
%load_ext autoreload
%autoreload 2

import os
from tqdm import tqdm_notebook as tqdm

import numpy as np
import tensorflow as tf
from tensorflow.keras import models, layers, losses, optimizers

from core.losses import pixelwise_mse
from core.utils import crop_div_32
from core.models import skip

import imageio
import matplotlib.pyplot as plt

In [ ]:
INPUT_IMG_PATH = "data/zebra_GT.png"
OUTPUT_IMG_NAME = os.path.splitext(os.path.basename(INPUT_IMG_PATH))[0]
OUTPUT_DIR = "output/super_resolution/"
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

img = imageio.imread(INPUT_IMG_PATH)
img = crop_div_32(img)
img = tf.image.convert_image_dtype(img, dtype=tf.float32)

In [ ]:
factor = 2 # The super-resolution image will have size of factor * original_size
input_dim = (img.shape[0] * factor, img.shape[1] * factor, 32)

num_filters_down = [128, 128, 128, 128, 128]
ksizes_down = [3, 3, 3, 3, 3]

num_filters_up = num_filters_down
ksizes_up = ksizes_down

num_filters_skip = [4, 4, 4, 4, 4]
ksizes_skip = [1, 1, 1, 1, 1]

sigma_p = 1/30
n_iter = 2000
lr = 0.01
upsampling_mode = "bilinear"


# Build the model
model = skip(input_dim, 
             num_filters_down, ksizes_down, 
             num_filters_up, ksizes_up,
             num_filters_skip, ksizes_skip,
             upsampling_mode, sigma_p=sigma_p,
             resize=img.shape[:2])

model.summary()

In [ ]:
from core.callbacks import SaveResultImage

z = np.random.uniform(0, 0.1, input_dim)
z = np.expand_dims(z, axis=0)
y_true = np.expand_dims(img, axis=0)

callbacks = [
    SaveResultImage(n=1, input_tensor=z, output_dir=OUTPUT_DIR, img_name=OUTPUT_IMG_NAME)
]
model.compile(loss=pixelwise_mse(),
              optimizer=optimizers.Adam(lr=lr))

model.fit(z, y_true, 
          epochs=n_iter,
          callbacks=callbacks)

In [ ]:
import matplotlib.pyplot as plt

sr = tf.image.resize(img, (img.shape[0] * factor, img.shape[1] * factor), method="bicubic")
imageio.imsave("data/sr_x2_bicubic_zebra.png", sr)